### Parsing Data


In [120]:
import sklearn as sk
from sklearn.linear_model import LogisticRegression
import numpy as np
import os
import re
import pickle

In [121]:
all_data_x = []
all_data_y = []

def parseToArray(fileName):
  file = open(fileName,"r", errors="replace")
  data_label = os.path.splitext(fileName)[0].split('/')[-1]
  content = file.read()
  samples = content.split("---")
  data_x = []
  for s in samples:
    X = []
    labels = s.strip().split('\n')
    for l in labels:
      for f in re.sub("[\s]", "",  l).split(','):
        X.append(int(f))
    data_x.append(X)
  data_y = [data_label] * len(data_x)
  return data_x, data_y
basedir = "./data/"
files  = [f for f in os.listdir(basedir) if os.path.isfile(os.path.join(basedir, f))]

for f in files:
  data_x,  data_y = parseToArray(basedir + f)
  all_data_x.extend(data_x)
  all_data_y.extend(data_y)


### Spliting Data

In [122]:
data = []
for i in range(len(all_data_x)):
  d = []
  d.extend(all_data_x[i])
  d.append(all_data_y[i])
  data.append(d)

data = sk.utils.shuffle(data)
n = len(data)

training_data = data[:int(n*0.8)]
training_data_x = [a[:-1] for a in training_data]
training_data_y = [a[-1] for a in training_data]

training_data_x = np.asarray(training_data_x)
training_data_y = np.asarray(training_data_y)

testing_data = data[int(n*0.8):]
testing_data_x = [a[:-1] for a in testing_data]
testing_data_y = [a[-1] for a in testing_data]

testing_data_x = np.asarray(testing_data_x)
testing_data_y = np.asarray(testing_data_y)

### Training The Model

In [123]:
model = LogisticRegression(multi_class='ovr',solver="liblinear")

model.fit(training_data_x, training_data_y)


LogisticRegression(multi_class='ovr', solver='liblinear')

### Testing

In [124]:
y_pred = model.predict(testing_data_x)
print(len(training_data_x))
print(sk.metrics.classification_report(testing_data_y,y_pred))
print(sk.metrics.accuracy_score(testing_data_y, y_pred))

220
              precision    recall  f1-score   support

    backward       0.90      0.86      0.88        21
     forward       0.88      0.95      0.91        22
  play-pause       1.00      0.92      0.96        13

    accuracy                           0.91        56
   macro avg       0.92      0.91      0.92        56
weighted avg       0.91      0.91      0.91        56

0.9107142857142857


### Saving The Model

In [119]:
filename = 'final_model.sav'
pickle.dump(model, open(filename, 'wb'))

### Load The Model


In [126]:
gestureInterpreter = pickle.load(open('final_model.sav', 'rb'))